<pre>
          ) ) )                           ( ( (
        ( ( (                               ) ) )
      ) ) )                                  ( ( (
   (~~~~~~~~~)                            (~~~~~~~~~)
    | POWER |                              | REWOP |
    | HOUSE |                              | ESUOH |   
    I      _._                            _._      I    
    I    /'   `\                        /`   '\    I    
    I   |   N   |                      |   N   |   I    
    f   |   |~~~~~~~~~~~~~~||~~~~~~~~~~~~~~|   |   f    
  .'    |   ||~~~~~~~~|    ||    |~~~~~~~~||   |    '.  
/'______|___||__###___|____||____|___###__||___|______'\


MITO
MOTI lity
quantifier

</pre>

In [ ]:
%run utils/experiment_chooser.ipynb
experiment_chooser.choose()

In [ ]:
from improc.mobility import calc_motility
from multiprocessing import Pool
from skimage.exposure import rescale_intensity
import tifffile
import numpy as np
import pathlib
import csv

experiment_base = experiment_chooser.fetch_base()
stacks = list((experiment_base / "processed_imgs" / "stacked").glob("*.tif"))
output_path =  pathlib.Path("/scratch/motion_hilighted")
if output_path is not None and not output_path.exists():
    output_path.mkdir()

def proc(stack_path):
    vertex = stack_path.name.strip(".tif")
    try:
        stack = tifffile.imread(stack_path)
    except:
        return None
    motility = calc_motility(stack)
    if output_path is not None:
        rescaled = rescale_intensity(motility["motion_hilighted"], out_range=np.uint8)
        tifffile.imwrite(output_path / f"{vertex}.tif", rescaled)
    return {
        "vertex": vertex,
        "counts_per_frame": motility["counts_per_frame"],
        "frac_motile": motility["pct"]
    }

rows = []
with Pool() as p:
    for idx, motile_row in enumerate(p.imap(proc, stacks)):
        print("\r", end="")
        print(f"{idx + 1} / {len(stacks)}", end="")
        if motile_row is not None:
            rows.append(motile_row)
    
with open(experiment_base / "results" / "mito_motility.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, ["vertex", "counts_per_frame", "frac_motile"])
    writer.writeheader()
    writer.writerows(rows)